In [55]:
from langchain_community.graphs import Neo4jGraph
from langchain.prompts.prompt import PromptTemplate
from langchain.chains import GraphCypherQAChain
from langchain_openai import ChatOpenAI, OpenAI

import os
import textwrap

In [56]:
# Load from environment
NEO4J_URI = os.getenv('NEO4J_URI')
NEO4J_USERNAME = os.getenv('NEO4J_USERNAME')
NEO4J_PASSWORD = os.getenv('NEO4J_PASSWORD')
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
NEO4J_DATABASE = os.getenv('NEO4J_DATABASE') or 'neo4j'

In [57]:
kg = Neo4jGraph(
    url=NEO4J_URI, username=NEO4J_USERNAME, password=NEO4J_PASSWORD
)

In [58]:
# Check the graph schema
kg.refresh_schema()
print(textwrap.fill(kg.schema, 60))

Node properties are the following: Movie {duration: STRING,
listed_in: STRING, country: STRING, cast: STRING,
date_added: STRING, director: STRING, release_year: STRING,
rating: STRING, description: STRING, id: STRING, title:
STRING, type: STRING},Person {name: STRING},Category {name:
STRING},Type {type: STRING},Country {name: STRING},Year
{value: INTEGER} Relationship properties are the following:
The relationships are the following: (:Movie)-[:TYPED_AS]->(
:Type),(:Movie)-[:IN_CATEGORY]->(:Category),(:Movie)-[:WHERE
]->(:Country),(:Movie)-[:CREATED_ON]->(:Year),(:Person)-[:AC
TED_IN]->(:Movie),(:Person)-[:WORK_WITH]->(:Person),(:Person
)-[:DIRECTED]->(:Movie),(:Year)-[:NEXT]->(:Year)


In [59]:
CYPHER_GENERATION_TEMPLATE = """Task:Generate Cypher statement to query a graph database.
Instructions:
Use only the provided relationship types and properties in the schema.
Do not use any other relationship types or properties that are not provided.
Schema:
{schema}
Note: Do not include any explanations or apologies in your responses.
Do not respond to any questions that might ask anything else than for you to construct a Cypher statement.
Do not include any text except the generated Cypher statement.
Examples: Here are a few examples of generated Cypher statements for particular questions:

# How many people starred in Top Gun?
MATCH (m:Movie {{title:"Top Gun"}})<-[:ACTED_IN]-()
RETURN count(*) AS numberOfActors

# What are the most popular movie categories
MATCH (m:Movie)-[:IN_CATEGORY]->(c:Category)
WITH c.name AS Category, COUNT(m) AS MoviesCount
RETURN Category, MoviesCount
ORDER BY MoviesCount DESC

# All the movies Keanu Reeves acted in
MATCH (p:Person {{name: "Keanu Reeves"}})-[:ACTED_IN]->(m:Movie)
RETURN m.title AS Movie, m.release_year AS ReleaseYear
ORDER BY m.release_year DESC

# all persons who have acted in at least one movie with Keanu Reeves
MATCH (p:Person {{name: "Keanu Reeves"}})-[:ACTED_IN]->(m:Movie)<-[:ACTED_IN]-(coActors)
WHERE p.name <> coActors.name
RETURN DISTINCT coActors.name AS CoActor

# Find pairs of actors who have worked together in more than one movie
MATCH (p1:Person)-[:ACTED_IN]->(m:Movie)<-[:ACTED_IN]-(p2:Person)
WHERE p1.name < p2.name
WITH p1, p2, COUNT(m) AS sharedMovies
WHERE sharedMovies > 1
RETURN p1.name AS Actor1, p2.name AS Actor2, sharedMovies
ORDER BY sharedMovies DESC

# Retrieve movies along with their categories.
MATCH (m:Movie)-[:IN_CATEGORY]->(c:Category)
RETURN m.title AS Movie, collect(c.name) AS Categories

# Find Movies Directed by Actors
MATCH (p:Person)-[:DIRECTED]->(m:Movie), (p)-[:ACTED_IN]->(m2:Movie)
RETURN p.name AS Person, m.title AS DirectedMovie, m2.title AS ActedInMovie

# Find movies released in the United States grouped by their release year
MATCH (m:Movie)-[:WHERE]->(c:Country {{name: "United States"}})
RETURN m.release_year AS ReleaseYear, collect(m.title) AS Movies
ORDER BY m.release_year

# Which actors have worked with Keanu Reeves in more than 2 movies and list the movies
MATCH (p1:Person)-[:ACTED_IN]->(m:Movie)<-[:ACTED_IN]-(p2:Person)
WHERE p1.name = "Keanu Reeves" AND p1.name <> p2.name
WITH p1, p2, collect(m.title) AS Movies, COUNT(m) AS sharedMovies
WHERE sharedMovies > 2
RETURN p2.name AS Actor, Movies
ORDER BY sharedMovies DESC

The question is:
{question}"""

In [60]:
CYPHER_GENERATION_PROMPT = PromptTemplate(
    input_variables=["schema", "question"], 
    template=CYPHER_GENERATION_TEMPLATE
)

In [67]:
cypherChain = GraphCypherQAChain.from_llm(
    OpenAI(temperature=0.9),
    graph=kg,
    verbose=True,
    validate_cypher= True,
    cypher_prompt=CYPHER_GENERATION_PROMPT,
    
)

In [69]:
cypherChain.invoke("Which actors have worked with Shah Rukh Khan in more than 2 movies and give me the count?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:


MATCH (p1:Person)-[:ACTED_IN]->(m:Movie)<-[:ACTED_IN]-(p2:Person)
WHERE p1.name = "Shah Rukh Khan" AND p1.name <> p2.name
WITH p1, p2, COUNT(m) AS sharedMovies
WHERE sharedMovies > 2
RETURN p2.name AS Actor, sharedMovies AS Count
ORDER BY sharedMovies DESC
Full Context:
[{'Actor': 'Boman Irani', 'Count': 5}, {'Actor': 'Naseeruddin Shah', 'Count': 4}, {'Actor': 'Tinnu Anand', 'Count': 4}, {'Actor': 'Juhi Chawla', 'Count': 4}, {'Actor': 'Satish Shah', 'Count': 3}, {'Actor': 'Tiku Talsania', 'Count': 3}, {'Actor': 'Anupam Kher', 'Count': 3}, {'Actor': 'Deepika Padukone', 'Count': 3}, {'Actor': 'Om Puri', 'Count': 3}]

> Finished chain.


{'query': 'Which actors have worked with Shah Rukh Khan in more than 2 movies and give me the count?',
 'result': ' Boman Irani, Naseeruddin Shah, Tinnu Anand, Juhi Chawla, Satish Shah, Tiku Talsania, Anupam Kher, Deepika Padukone, and Om Puri have all worked with Shah Rukh Khan in more than 2 movies. The count for each actor is as follows: Boman Irani (5), Naseeruddin Shah (4), Tinnu Anand (4), Juhi Chawla (4), Satish Shah (3), Tiku Talsania (3), Anupam Kher (3), Deepika Padukone (3), and Om Puri (3).'}

In [70]:
cypherChain.invoke("Which movies star Tinnu Anand and Shah Rukh Khan?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:

MATCH (m:Movie)<-[:ACTED_IN]-(p1:Person {name: "Tinnu Anand"}), (m)<-[:ACTED_IN]-(p2:Person {name: "Shah Rukh Khan"})
RETURN m.title AS Movie, p1.name AS TinnuAnand, p2.name AS ShahRukhKhan
Full Context:
[{'Movie': 'Chamatkar', 'TinnuAnand': 'Tinnu Anand', 'ShahRukhKhan': 'Shah Rukh Khan'}, {'Movie': 'Ram Jaane', 'TinnuAnand': 'Tinnu Anand', 'ShahRukhKhan': 'Shah Rukh Khan'}, {'Movie': 'Trimurti', 'TinnuAnand': 'Tinnu Anand', 'ShahRukhKhan': 'Shah Rukh Khan'}, {'Movie': 'Oh Darling Yeh Hai India', 'TinnuAnand': 'Tinnu Anand', 'ShahRukhKhan': 'Shah Rukh Khan'}]

> Finished chain.


{'query': 'Which movies star Tinnu Anand and Shah Rukh Khan?',
 'result': '\nTinnu Anand and Shah Rukh Khan have starred in several movies together, such as Chamatkar, Ram Jaane, Trimurti, and Oh Darling Yeh Hai India.'}